## 文本分类

模型 预测

常用模型
    DAN : 情感分类
    双向RNN:
    堆叠神经网络
    CNN:



使用PyTorch模型和TorchText做情感分析
检测一段文字的情感是证明还是负面的。数据集：电影评论IMDb数据集

模型从简单到复杂：
   * Word Averaging 模型
   * RNN / LSTM 模型
   * CNN 模型

### 准备数据
* TorchText中的Field，决定数据怎么被处理
* 使用Text Field来处理定义评论，使用Label field处理2中情感类型pos， neg
* Text Field中tokenize=‘spacy' 表示我们用SpaCy tokenizer来tokenize英文语句。如果不使用该参数，默认使用空格来分词
* 安装SpaCy   `pip install spacy & python -m spacy donwload en
* label 有LabelField定义


In [1]:
import torch
from torchtext import data
from torchtext.data import Field,Example,Dataset

SEED = 1
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.utils(tokenize='spacy')
LABEL = data.LabelField(dtype=torch.float)

AttributeError: module 'torchtext.data' has no attribute 'Field'

In [ ]:
from torchtext import datasets
train_data, test_data = datasets.IMDB.splits(TEXT, LABEL) # 自动下载IMDB数据集 5w电影评论

In [ ]:
BATCH_SIZE = 64
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


## Word Averaging 模型
